In [ ]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re
import os  # for file paths
import awswrangler as wr

# Disposals Outputs

#### Child counts

In [ ]:
pydb.create_temp_table( 
f"""
SELECT
  Year,
  Quarter,
  'Disposal' AS Type,
  'Children' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
   Disp_type AS Disposal_type,
   order_desc AS Order_type,
   order_code AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_disps_all_children_d
Where
  Year>=2011
Group BY
  Year,
  Quarter,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End,
  Disp_type,
  order_desc,
  order_code
  
""",

"child_counts_d")

### Order counts

In [ ]:
pydb.create_temp_table( 
f"""
SELECT
  Year,
  Quarter,
  'Disposal' AS Type,
  'Order type' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
   Disp_type AS Disposal_type,
   order_desc AS Order_type,
   order_code AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_all_disposals_d
Where
  Year>=2011
Group BY
  Year,
  Quarter,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End,
  Disp_type,
  order_desc,
  order_code
  
""",

"order_type_counts_d")

### Case counts

In [ ]:
pydb.create_temp_table( 
f"""
SELECT
  Year,
  Quarter,
  'Cases' AS Type,
  'Cases closed' AS Count_type,
  CASE WHEN case_type = 'C' THEN 'Public law'
       WHEN case_type = 'P' THEN 'Private law'
    End AS Public_private,
   'N/A' AS Disposal_type,
   'All' AS Order_type,
  CAST(NULL AS INTEGER) AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_closed_cases_d
Where
  Year>=2011
Group BY
  Year,
  Quarter,
  CASE WHEN case_type = 'C' THEN 'Public law'
       WHEN case_type = 'P' THEN 'Private law'
    End
  
""",

"disp_case_counts_d")

### Combined Special Guardianship and Supervision orders (this code will need revisiting once duplicate issues have been resolved)

In [ ]:
#Supervision orders
pydb.create_temp_table( 
f"""
SELECT * FROM __temp__.ca_disps_all_children_d WHERE order_code = 4   
""",
"so")

In [ ]:
#SGO orders
pydb.create_temp_table( 
f"""
SELECT * FROM __temp__.ca_disps_all_children_d WHERE order_code = 27   
""",
"sgo")

In [ ]:
#Join SO and SGO info together to see if both orders were given to the same child on the same day
pydb.create_temp_table( 
f"""
SELECT
  so.*,
  sgo.order_desc AS sgo_order
FROM
  __temp__.so so 
  LEFT JOIN __temp__.sgo sgo 
    ON so.receipt_date = sgo.receipt_date
    AND so.child_role_id = sgo.child_role_id
WHERE
  sgo.order_desc IS NOT NULL
""",

"so_sgo_children_d")

In [ ]:
# now get rid of the children and count only orders
pydb.create_temp_table( 
f"""
SELECT
  DISTINCT year,
           quarter,
           order_case_type,
           disp_type_code,
           event
FROM
  __temp__.so_sgo_children_d

""",

"so_sgo_orders_d")

In [ ]:
# csv data for combined orders - children
pydb.create_temp_table( 
f"""
SELECT
  Year,
  Quarter,
  'Disposal' AS Type,
  'Combined orders - children' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
   'Combined orders' AS Disposal_type,
   'Supervision and SGO' AS Order_type,
  CAST(NULL AS INTEGER) AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.so_sgo_children_d
Where
  Year>=2011
  AND disp_type_code IN (1,2) 
Group BY
  Year,
  Quarter,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End
  
""",

"combined_orders_CH_d")

In [ ]:
#csv data for combined orders - orders made only
pydb.create_temp_table( 
f"""
SELECT
  Year,
  Quarter,
  'Disposal' AS Type,
  'Combined orders - orders' AS Count_type,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End AS Public_private,
   'Combined orders' AS Disposal_type,
   'Supervision and SGO' AS Order_type,
  CAST(NULL AS INTEGER) AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.so_sgo_orders_d
Where
  Year>=2011
  AND disp_type_code IN (1,2) 
Group BY
  Year,
  Quarter,
  CASE WHEN order_case_type = 'C' THEN 'Public law'
       WHEN order_case_type = 'P' THEN 'Private law'
    End
  
""",

"combined_orders_ORD_d")

### Combine all datasets

In [ ]:
pydb.create_temp_table( 
f"""
SELECT
  *
FROM
  __temp__.child_counts_d
UNION ALL
SELECT
  *
FROM
  __temp__.order_type_counts_d
UNION ALL
SELECT
  *
FROM
  __temp__.disp_case_counts_d
UNION ALL 
SELECT
  *
FROM
  __temp__.combined_orders_CH_d
UNION ALL 
SELECT
  *
FROM
  __temp__.combined_orders_ORD_d
  
""",

"disp_csv_d")

### Exporting data

In [ ]:
data = pydb.read_sql_query ("select * from __temp__.disp_csv_d")

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv (r's3://alpha-family-data/fcsq_processing/CA_disps/test_csv/ca_disps_d.csv', header = True)

In [ ]:
df.to_csv ('Exports/disposals.csv',header = True)